# **Dunder or Magic Methods**

#### Python Magic methods are the methods starting and ending with **double underscores ‘__’**. They are defined by built-in classes in Python and commonly used for operator overloading. 

#### They are also called Dunder methods, *Dunder here means “Double Under (Underscores)”.*



#### => The __del__ method is a special method in Python classes that is called when an **object is about to be destroyed**, typically as part of the garbage collection process.

#### => The __repr__ method in Python is a special method used to define a **string** representation of an object. 

In [46]:
class Vector:
    def __init__(self,x,y):
        self.x = x 
        self.y = y
    
    def __add__(self,other):
        return Vector(self.x + other.x, self.y + other.y)
    
    def __repr__(self):
        return (f"X : {self.x}, Y : {self.y}")
    
    def __del__(self):
        print(f"Object is being destroyed")
    

In [47]:
v1 = Vector(x=10, y=20)
v2 = Vector(x=30, y=40)

v3 = v1 + v2
print(v3)

Object is being destroyed
Object is being destroyed
X : 40, Y : 60


In [48]:
del v3
print(v3)

Object is being destroyed


NameError: name 'v3' is not defined

In [49]:
class Counter:
    def __init__(self):
        self.count = 0

    def __call__(self):
        self.count += 1
        return self.count


In [50]:
# Creating an instance of Counter
counter = Counter()

# Calling the instance like a function increments the count
print(counter())  # Prints 1
print(counter())  # Prints 2
print(counter())  # Prints 3


1
2
3


## **__getitem__** and **__len__**

### In Python, **__getitem__(self,index)** is a special method that allows an object to be indexed, just like a sequence **(e.g., list, tuple) or a mapping (e.g., dictionary)**.

In [28]:
class MyClass:
    def __init__(self, data):
        self.data = data
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        return self.data[index]



In [31]:
# Creating an instance of MyClass
my_object = MyClass([1, 2, 3, 4, 5])

# Using __getitem__ to access elements
print(f"Data = {my_object[2]}") 
print(f"Object Len = {len(my_object)}")

Data = 3
Object Len = 5


## Example

In [25]:
import torch
from torch.utils.data import Dataset

class Flickr27Dataset(Dataset):
    def __init__(self, root_folder, annotation_file, class_name_to_label ,transform=None):
        self.root_folder = root_folder
        self.annotations = annotation_file
        self.transform = transform
        self.ClasstoLabel = class_name_to_label
        
    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, idx):
        image_name = self.annotations.iloc[idx][0]
        image_path = f"{self.root_folder}/{image_name}"

        # Load image
        image = Image.open(image_path).convert("RGB")
        wt, ht = image.size
        image = image.resize((config.WIDTH, config.HEIGHT))
        image = np.array(image).astype(np.float32)
        image /= 255.0
        
        
        # Extract bounding box coordinates and class label
        xmin = self.annotations.iloc[idx][3]
        ymin = self.annotations.iloc[idx][4]
        xmax = self.annotations.iloc[idx][5]
        ymax = self.annotations.iloc[idx][6]
        
        # change bounding box as per resize
        
        xmin_corr = (xmin/wt)*config.WIDTH
        xmax_corr = (xmax/wt)*config.WIDTH
        ymin_corr = (ymin/ht)*config.HEIGHT
        ymax_corr = (ymax/ht)*config.HEIGHT
        
        
        box = [xmin_corr, ymin_corr, xmax_corr, ymax_corr]
        box = [float(coord) for coord in box]
        class_name = self.annotations.iloc[idx][1]
        class_label = self.ClasstoLabel[class_name]
         
        
        # Create target dictionary containing bounding box and class information
        target = {
            'image_id' : torch.tensor([idx]),
            'boxes' : torch.tensor([box], dtype=torch.float32),
            'labels' : torch.tensor([class_label], dtype=torch.int64),
        }
        
        if self.transform:
            sample = {
                'image': image,
                'bboxes': target['boxes'],
                'labels': target['labels']
            }
            sample = self.transform(**sample)
            image = sample['image']
            
            target['boxes'] = torch.as_tensor(sample['bboxes'])
 

        return image, target



AttributeError: module 'torch.nn' has no attribute 'Dataset'